In [1]:
import math
from numba import njit, prange, cuda
import numpy as np
import random
import time
from numba.cuda.random import (create_xoroshiro128p_states,xoroshiro128p_uniform_float32)


@cuda.jit
def random_2d(arr, rng_states):
    # Per-dimension thread indices and strides
    startx, starty = cuda.grid(2)
    stridex, stridey= cuda.gridsize(2)

    # Linearized thread index
    tid = (stridey * stridex) + (starty * stridex) + startx

    # Use strided loops over the array to assign a random value to each entry
    for j in range(starty, arr.shape[0], stridey):
        for k in range(startx, arr.shape[1], stridex):
            arr[j, k] = xoroshiro128p_uniform_float32(rng_states, tid)


def matrix_multiply(A, B):
    n = A.shape[0]
    m = B.shape[1]
    result = np.zeros((n, m))

    for i in range(n):
        for j in range(m):
            for k in range(A.shape[1]):
                result[i][j] += A[i][k] * B[k][j]
    return result





@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    # Needs to be 2 per docs: https://numba.pydata.org/numba-doc/latest/cuda/kernels.html#absolute-positions
    i, k = cuda.grid(2)
    if k < A.shape[0] and i < C.shape[0]:
        tmp = A[i, k]
        for j in range(C.shape[1]):
             C[i, j] += tmp * B[k, j]

# input two matrices of size n x m
size = 500

#Memory issues: https://stackoverflow.com/questions/43945820/matrix-multiplication-in-cuda-running-out-of-memory


# Array dimensions
X, Y= size, size

# Block and grid dimensions
bx, by = 8, 8
gx, gy = 16, 16

# Total number of threads
nthreads = bx * by * gx * gy

# Initialize a state for each thread
rng_states = create_xoroshiro128p_states(nthreads, seed=1)
# Generate random numbers
matrix1 = cuda.device_array((X, Y), dtype=np.float16)
random_2d[(gx, gy), (bx, by)](matrix1, rng_states)
matrix2 = cuda.device_array((X, Y), dtype=np.float16)
random_2d[(gx, gy), (bx, by)](matrix2, rng_states)
res = cuda.device_array((X, Y), dtype=np.float16)


#https://oneflow2020.medium.com/how-to-choose-the-grid-size-and-block-size-for-a-cuda-kernel-d1ff1f0a7f92
# max blocks per docs for A100: https://docs.nvidia.com/cuda/ampere-tuning-guide/index.html#:~:text=The%20maximum%20number%20of%20thread,GPUs%20with%20compute%20capability%208.6.
threadsperblock = (32,32)
blockspergrid_x = math.ceil(matrix1.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(matrix1.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
# threadsperblock = 32
# blockspergrid = (matrix1.size + (threadsperblock - 1)) // threadsperblock
start = time.time_ns()
matmul[blockspergrid, threadsperblock](matrix1, matrix2, res)
end = time.time_ns()

time_diff = end-start
time_diff_s = time_diff / (10 ** 9) # convert to floating-point seconds
print("Time taken for GPU {0:.6f} seconds".format(time_diff_s))


matrix1 = np.random.rand(size, size).astype(np.float16)
matrix2 = np.random.rand(size, size).astype(np.float16)

start = time.time()
result = matrix_multiply(matrix1, matrix2)
end = time.time()

time_diff = end - start
print("Time taken for CPU (Python) matrix multiplication: {:.6f} seconds".format(time_diff))



CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: